In [1]:
import dask.dataframe as dd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 771 kB 4.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-linux_x86_64.whl size=2626491 sha256=baf33b7a9cecb536c7061bec79b202292fb448bcf917148f631e2ea55ea0a520
  Stored in directory: /root/.cache/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


In [38]:
# Libraries for data preparation & visualization
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "png"

# Ignore printing warnings for general readability
import warnings 
warnings.filterwarnings("ignore")

# pip install scikit-surprise
# Importing libraries for model building & evaluation
from sklearn.model_selection import train_test_split
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD
from surprise import accuracy


In [39]:
# Loading the dataset 
def loaddata(filename):
    df = pd.read_csv(f'{filename}.csv',sep=';',error_bad_lines=False,warn_bad_lines=False,encoding='latin-1')
    return df

book   = loaddata("Books")
user   = loaddata("Users")
rating = loaddata("Ratings")


In [40]:
rating.head(3)

,"User-ID,ISBN,Book-Rating"
0,"276725,034545104X,0"
1,"276726,0155061224,5"
2,"276727,0446520802,0"


In [41]:
rating.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 1 columns):
 #   Column                    Non-Null Count    Dtype 
---  ------                    --------------    ----- 
 0   User-ID,ISBN,Book-Rating  1149780 non-null  object
dtypes: object(1)
memory usage: 8.8+ MB


In [42]:
print(f'Duplicate entries: {rating.duplicated().sum()}')


Duplicate entries: 0


In [43]:
pip install kaleido

Note: you may need to restart the kernel to use updated packages.


In [45]:

# Let's visualize the 'Book-Rating' column

rating_types = rating['Book-Rating'].value_counts()
x,y   = rating_types.index, rating_types.values
data  = go.Bar(x=x,y=y, text=y, textposition="outside")
Layout=go.Layout(title="Count of ratings",xaxis={'title':'Rating'},yaxis={'title':'Count'})
go.Figure(data,Layout)


KeyError: 'Book-Rating'

In [6]:
#reader = Reader(rating_scale=(0, 10))
#data   = Dataset.load_from_df(rating[['User-ID','ISBN','Book-Rating']], reader)


# Split the data into training & testing sets. Python's surprise documentation has the steps detailed out
# https://surprise.readthedocs.io/en/stable/FAQ.html

raw_ratings = rating
import random
#random.shuffle(raw_ratings)                 # shuffle dataset

threshold   = int(len(raw_ratings)*0.8)

trainset = raw_ratings[:threshold] # 80% of data is trainset
testset = raw_ratings[threshold:] # 20% of data is testset




In [21]:
pip install dask

     ---------------------------------------- 1.1/1.1 MB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 139.5/139.5 kB ? eta 0:00:00
     ---------------------------------------- 55.8/55.8 kB ? eta 0:00:00
     ---------------------------------------- 151.6/151.6 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [28]:


#rating = pd.merge(rating,book, on="ISBN")[['User-ID','Book-Title','Book-Rating']] # merging with the book dataframe
#rating   

data=pd.read_csv('Books.csv')
data1=pd.read_csv('Ratings.csv')                                                        # on 'ISBN' to get 'Book-Title'
rating = pd.merge(data, data1, on="ISBN")


In [31]:
rating = rating[rating['User-ID'].isin(rating_users[rating_users['Rating']>250]['index'])]
rating = rating[rating['ISBN'].isin(rating_books[rating_books['Rating']> 50]['index'])]

rating


NameError: name 'rating_users' is not defined

In [29]:
# creating a surprise object

reader = Reader(rating_scale=(0, 10))
data   = Dataset.load_from_df(rating[['User-ID','Book-Title','Book-Rating']], reader)


# Split the data into training & testing sets. Python's surprise documentation has the steps detailed out
# https://surprise.readthedocs.io/en/stable/FAQ.html

raw_ratings = data.raw_ratings
import random
random.shuffle(raw_ratings)                 # shuffle dataset

threshold   = int(len(raw_ratings)*0.8)

train_raw_ratings = raw_ratings[:threshold] # 80% of data is trainset
test_raw_ratings  = raw_ratings[threshold:] # 20% of data is testset

data.raw_ratings = train_raw_ratings        # data is now the trainset
trainset         = data.build_full_trainset() 
testset          = data.construct_testset(test_raw_ratings)

In [30]:
# Model fit & prediction - KNNWithMeans

sim_options = {'name':'cosine','min_support':3,'user_based':False}
final_model = KNNWithMeans(sim_options=sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')



Computing the cosine similarity matrix...


MemoryError: Unable to allocate 340. GiB for an array with shape (213700, 213700) and data type float64

In [10]:
# Model fit & prediction - KNNWithMeans
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD
sim_options = {'name':'cosine','min_support':3,'user_based':False}
final_model = KNNWithMeans(sim_options=sim_options)

# Fitting the model on trainset & predicting on testset, printing test accuracy
pred = final_model.fit(trainset).test(testset)

print(f'\nUnbiased Testing Performance:')
print(f'MAE: {accuracy.mae(pred)}, RMSE: {accuracy.rmse(pred)}')


Computing the cosine similarity matrix...


MemoryError: Unable to allocate 340. GiB for an array with shape (213653, 213653) and data type float64

In [ ]:
# Let's visualize ratings received by books

rating_books = rating['ISBN'].value_counts().reset_index().\
               rename({'Index':'ISBN','ISBN':'Rating'}, axis=1)

data  = go.Scatter(x = rating_books.index, y= rating_books['Rating'])
Layout= go.Layout(title="Ratings received per book",xaxis={'title':'Book Count'}, yaxis={'title':'Ratings'})
go.Figure(data, Layout)      